# LangChain EnsembleRetriever Quick Reference

## Introduction

The **EnsembleRetriever** is a powerful retrieval mechanism within the LangChain framework designed to enhance information retrieval by combining the results of multiple retrievers. By leveraging the strengths of different retrieval algorithms, the EnsembleRetriever achieves superior performance compared to using a single retriever. This approach, often referred to as "hybrid search," integrates sparse retrievers (e.g., BM25) and dense retrievers (e.g., Chroma or FAISS) to provide a more comprehensive and accurate retrieval system.

### Key Features
- **Combination of Multiple Retrievers**: The EnsembleRetriever integrates various retrieval methods, such as sparse and dense retrievers, to handle diverse query types and document structures.
- **Reranking Mechanism**: It uses the **Reciprocal Rank Fusion (RRF)** algorithm to rerank results from individual retrievers, ensuring the most relevant documents are prioritized.
- **Improved Performance**: By combining keyword-based and semantic-based retrieval, the EnsembleRetriever delivers better accuracy and relevance, especially in complex search scenarios.
- **Customizable Weights**: Users can assign weights to individual retrievers to prioritize specific retrieval methods based on their strengths.
- **Runtime Configuration**: The retriever supports dynamic configuration, allowing users to adjust parameters (e.g., the number of documents to retrieve) at runtime.

### Use Cases
1. **Hybrid Search**: Combining sparse and dense retrievers to handle both keyword-based and semantic-based queries effectively.
2. **Domain-Specific Retrieval**: Integrating custom retrievers for specialized domains (e.g., legal, medical, or scientific documents).
3. **Metadata Filtering**: Refining search results by filtering documents based on metadata (e.g., source, date, or category).
4. **Dynamic Query Handling**: Adjusting retrieval strategies at runtime to adapt to different query types or user preferences.
5. **Enhanced Search Relevance**: Improving search relevance in applications like chatbots, recommendation systems, and knowledge bases.

### Comparison Table: Sparse vs. Dense vs. Hybrid Retrieval

| Feature                | Sparse Retrieval (e.g., BM25)       | Dense Retrieval (e.g., Chroma)      | Hybrid Retrieval (EnsembleRetriever) |
|------------------------|-------------------------------------|-------------------------------------|--------------------------------------|
| **Strengths**          | Keyword-based matching              | Semantic similarity matching        | Combines both keyword and semantic   |
| **Weaknesses**         | Struggles with semantic queries     | Struggles with exact keyword match  | Requires more computational resources|
| **Use Cases**          | Keyword-heavy queries               | Semantic-heavy queries              | Complex queries requiring both       |
| **Performance**        | Fast for exact keyword searches     | Slower but more accurate for semantics | Balanced performance for hybrid tasks|
| **Customization**      | Limited to keyword-based tuning     | Limited to embedding-based tuning   | Highly customizable with weights     |

---

## Preparation

### Installing Required Libraries
This section installs the necessary Python libraries for working with LangChain, OpenAI embeddings, and Chroma vector store. These libraries include:
- `langchain-openai`: Provides integration with OpenAI's embedding models.
- `langchain_community`: Contains community-contributed modules and tools for LangChain.
- `langchain_experimental`: Includes experimental features and utilities for LangChain.
- `langchain-chroma`: Enables integration with the Chroma vector database.
- `chromadb`: The core library for the Chroma vector database.

In [ ]:
!pip install -qU langchain-openai
!pip install -qU langchain_community
!pip install -qU langchain_experimental
!pip install -qU langchain-chroma>=0.1.2
!pip install -qU chromadb
!pip install -qU rank_bm25

### Initializing OpenAI Embeddings
This section demonstrates how to securely fetch an OpenAI API key using Kaggle's `UserSecretsClient` and initialize the OpenAI embedding model. The `OpenAIEmbeddings` class is used to create an embedding model instance, which will be used to convert text into numerical embeddings.

Key steps:
1. **Fetch API Key**: The OpenAI API key is securely retrieved using Kaggle's `UserSecretsClient`.
2. **Initialize Embeddings**: The `OpenAIEmbeddings` class is initialized with the `text-embedding-3-small` model and the fetched API key.

This setup ensures that the embedding model is ready for use in downstream tasks, such as caching embeddings or creating vector stores.

In [ ]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from kaggle_secrets import UserSecretsClient

# Fetch API key securely
user_secrets = UserSecretsClient()
my_api_key = user_secrets.get_secret("api-key-openai")

# Initialize OpenAI embeddings
embed = OpenAIEmbeddings(model="text-embedding-3-small", api_key=my_api_key)
model = ChatOpenAI(model="gpt-4o-mini", temperature=1.0, api_key=my_api_key)

---

## 1. Document Retrieval and Management

### Example 1: Basic Ensemble Retrieval
This example demonstrates how to initialize an `EnsembleRetriever` with a BM25 retriever and a Chroma vector store retriever, and then retrieve documents for a query.

In [ ]:
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_chroma import Chroma

# Sample documents
doc_list_1 = [
    "I like apples",
    "I like oranges",
    "Apples and oranges are fruits",
]

doc_list_2 = [
    "You like apples",
    "You like oranges",
]

# Initialize BM25 retriever
bm25_retriever = BM25Retriever.from_texts(
    doc_list_1, metadatas=[{"source": 1}] * len(doc_list_1)
)
bm25_retriever.k = 2

# Initialize Chroma vector store retriever
chroma_vectorstore = Chroma.from_texts(
    doc_list_2, embed, metadatas=[{"source": 2}] * len(doc_list_2)
)
chroma_retriever = chroma_vectorstore.as_retriever(search_kwargs={"k": 2})

# Initialize EnsembleRetriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, chroma_retriever], weights=[0.5, 0.5]
)

# Retrieve documents
docs = ensemble_retriever.invoke("apples")
print(docs)

### Example 2: Customizing Retrieval Parameters
This example shows how to customize the retrieval parameters, such as the number of documents to retrieve (`k`) and the weights assigned to each retriever.

In [ ]:
# Update BM25 retriever to return top 3 documents
bm25_retriever.k = 3

# Update Chroma retriever to return top 3 documents
chroma_retriever = chroma_vectorstore.as_retriever(search_kwargs={"k": 3})

# Reinitialize EnsembleRetriever with updated weights
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, chroma_retriever], weights=[0.7, 0.3]
)

# Retrieve documents
docs = ensemble_retriever.invoke("oranges")
print(docs)

---

## 2. Batch Processing

### Example 1: Batch Retrieval
This example demonstrates how to use the `batch` method to retrieve documents for multiple queries in parallel.

In [ ]:
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_chroma import Chroma

# Sample documents
doc_list_1 = [
    "I like apples",
    "I like oranges",
    "Apples and oranges are fruits",
]

doc_list_2 = [
    "You like apples",
    "You like oranges",
]

# Initialize BM25 retriever
bm25_retriever = BM25Retriever.from_texts(
    doc_list_1, metadatas=[{"source": 1}] * len(doc_list_1)
)
bm25_retriever.k = 2

# Initialize Chroma vector store retriever
chroma_vectorstore = Chroma.from_texts(
    doc_list_2, embed, metadatas=[{"source": 2}] * len(doc_list_2)
)
chroma_retriever = chroma_vectorstore.as_retriever(search_kwargs={"k": 2})

# Initialize EnsembleRetriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, chroma_retriever], weights=[0.5, 0.5]
)

# Define multiple queries
queries = ["apples", "oranges", "fruits"]

# Retrieve documents for all queries in batch
batch_docs = ensemble_retriever.batch(queries)
for query, docs in zip(queries, batch_docs):
    print(f"Query: {query}")
    print(docs)

### Example 2: Batch Retrieval with Custom Configuration
This example shows how to apply custom configurations (e.g., adjusting `k` for the Chroma retriever) during batch retrieval.

In [ ]:
# Define a custom configuration for the Chroma retriever
config = {"configurable": {"search_kwargs": {"k": 1}}}

# Retrieve documents for all queries in batch with custom configuration
batch_docs = ensemble_retriever.batch(queries, config=config)
for query, docs in zip(queries, batch_docs):
    print(f"Query: {query}")
    print(docs)

---

## 3. Streaming

### Example 1: Streaming Retrieval Results
This example demonstrates how to stream retrieval results for a query.

In [ ]:
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_chroma import Chroma

# Sample documents
doc_list_1 = [
    "I like apples",
    "I like oranges",
    "Apples and oranges are fruits",
]

doc_list_2 = [
    "You like apples",
    "You like oranges",
]

# Initialize BM25 retriever
bm25_retriever = BM25Retriever.from_texts(
    doc_list_1, metadatas=[{"source": 1}] * len(doc_list_1)
)
bm25_retriever.k = 2

# Initialize Chroma vector store retriever
chroma_vectorstore = Chroma.from_texts(
    doc_list_2, embed, metadatas=[{"source": 2}] * len(doc_list_2)
)
chroma_retriever = chroma_vectorstore.as_retriever(search_kwargs={"k": 2})

# Initialize EnsembleRetriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, chroma_retriever], weights=[0.5, 0.5]
)

# Stream documents for a query
for doc in ensemble_retriever.stream("apples"):
    print(doc)

### Example 2: Customizing Streaming Behavior
This example shows how to customize the streaming behavior by adjusting the number of documents streamed.

In [ ]:
# Update Chroma retriever to return only 1 document during streaming
chroma_retriever = chroma_vectorstore.as_retriever(search_kwargs={"k": 1})

# Reinitialize EnsembleRetriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, chroma_retriever], weights=[0.5, 0.5]
)

# Stream documents for a query
for doc in ensemble_retriever.stream("oranges"):
    print(doc)

---

## 4. Error Handling and Retries

### Example 1: Adding Fallback Retrievers
This example demonstrates how to add fallback retrievers to handle failures.

In [ ]:
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_chroma import Chroma
from langchain_core.runnables import RunnableLambda

# Sample documents
doc_list_1 = [
    "I like apples",
    "I like oranges",
    "Apples and oranges are fruits",
]

doc_list_2 = [
    "You like apples",
    "You like oranges",
]

# Initialize BM25 retriever
bm25_retriever = BM25Retriever.from_texts(
    doc_list_1, metadatas=[{"source": 1}] * len(doc_list_1)
)
bm25_retriever.k = 2

# Initialize Chroma vector store retriever
chroma_vectorstore = Chroma.from_texts(
    doc_list_2, embed, metadatas=[{"source": 2}] * len(doc_list_2)
)
chroma_retriever = chroma_vectorstore.as_retriever(search_kwargs={"k": 2})

# Initialize EnsembleRetriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, chroma_retriever], weights=[0.5, 0.5]
)

# Define a fallback retriever
fallback_retriever = RunnableLambda(lambda x: [{"page_content": "Fallback document", "metadata": {"source": "fallback"}}])

# Add fallback to the ensemble retriever
ensemble_retriever_with_fallback = ensemble_retriever.with_fallbacks([fallback_retriever])

# Retrieve documents (fallback will be used if primary retrievers fail)
docs = ensemble_retriever_with_fallback.invoke("unknown query")
print(docs)

### Example 2: Retrying on Failure
This example shows how to configure the retriever to retry on specific exceptions.

In [ ]:
# Configure retries for the ensemble retriever
ensemble_retriever_with_retry = ensemble_retriever.with_retry(
    retry_if_exception_type=(ValueError,), stop_after_attempt=3
)

# Retrieve documents (retries will be attempted on failure)
docs = ensemble_retriever_with_retry.invoke("apples")
print(docs)

---

## 5. Lifecycle Listeners

### Example 1: Adding Lifecycle Listeners
This example demonstrates how to add lifecycle listeners to the retriever.

In [ ]:
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_chroma import Chroma

# Sample documents
doc_list_1 = [
    "I like apples",
    "I like oranges",
    "Apples and oranges are fruits",
]

doc_list_2 = [
    "You like apples",
    "You like oranges",
]

# Initialize BM25 retriever
bm25_retriever = BM25Retriever.from_texts(
    doc_list_1, metadatas=[{"source": 1}] * len(doc_list_1)
)
bm25_retriever.k = 2

# Initialize Chroma vector store retriever
chroma_vectorstore = Chroma.from_texts(
    doc_list_2, embed, metadatas=[{"source": 2}] * len(doc_list_2)
)
chroma_retriever = chroma_vectorstore.as_retriever(search_kwargs={"k": 2})

# Initialize EnsembleRetriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, chroma_retriever], weights=[0.5, 0.5]
)

# Define lifecycle listeners
def on_start(run_obj):
    print(f"Retrieval started with input: {run_obj.inputs}")

def on_end(run_obj):
    print(f"Retrieval ended with output: {run_obj.outputs}")

# Add lifecycle listeners to the ensemble retriever
ensemble_retriever_with_listeners = ensemble_retriever.with_listeners(
    on_start=on_start, on_end=on_end
)

# Retrieve documents (listeners will be triggered)
docs = ensemble_retriever_with_listeners.invoke("oranges")
print(docs)

### Example 2: Customizing Listener Behavior
This example shows how to customize listener behavior by adding metadata to the run.

In [ ]:
def on_start_with_metadata(run_obj):
    print(f"Retrieval started with input: {run_obj.inputs} and metadata: {run_obj.metadata}")

# Add lifecycle listeners with metadata
ensemble_retriever_with_listeners = ensemble_retriever.with_listeners(
    on_start=on_start_with_metadata, on_end=on_end
)

# Retrieve documents (listeners will be triggered with metadata)
docs = ensemble_retriever_with_listeners.invoke("apples", config={"metadata": {"user": "test"}})
print(docs)

---

## 6. Configuration and Binding

### Example 1: Binding Configuration at Runtime
This example demonstrates how to bind configuration (e.g., adjusting `k` for the Chroma retriever) at runtime.

In [ ]:
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_chroma import Chroma
from langchain_core.runnables import ConfigurableField

# Sample documents
doc_list_1 = [
    "I like apples",
    "I like oranges",
    "Apples and oranges are fruits",
]

doc_list_2 = [
    "You like apples",
    "You like oranges",
]

# Initialize BM25 retriever
bm25_retriever = BM25Retriever.from_texts(
    doc_list_1, metadatas=[{"source": 1}] * len(doc_list_1)
)
bm25_retriever.k = 2

# Initialize Chroma vector store retriever
chroma_vectorstore = Chroma.from_texts(
    doc_list_2, embed, metadatas=[{"source": 2}] * len(doc_list_2)
)
chroma_retriever = chroma_vectorstore.as_retriever(
    search_kwargs={"k": 2}
).configurable_fields(
    search_kwargs=ConfigurableField(
        id="search_kwargs_chroma",
        name="Search Kwargs",
        description="The search kwargs to use",
    )
)

# Initialize EnsembleRetriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, chroma_retriever], weights=[0.5, 0.5]
)

# Bind configuration at runtime
config = {"configurable": {"search_kwargs_chroma": {"k": 1}}}
docs = ensemble_retriever.invoke("oranges", config=config)
print(docs)

### Example 2: Binding Arguments to the Retriever
This example shows how to bind additional arguments to the retriever.

In [ ]:
# Bind additional arguments to the retriever
ensemble_retriever_with_args = ensemble_retriever.bind(k=1)

# Retrieve documents with bound arguments
docs = ensemble_retriever_with_args.invoke("apples")
print(docs)

---

## Best Practices

The **Best Practices** examples demonstrate how to effectively use the `EnsembleRetriever` in LangChain to combine multiple retrieval techniques for improved document retrieval. These examples highlight key features and configurations, such as:

1. **Hybrid Search**: Combining sparse (keyword-based) and dense (semantic-based) retrievers to leverage their complementary strengths.
2. **Runtime Configuration**: Dynamically adjusting retriever parameters (e.g., the number of documents to retrieve) at runtime.
3. **Custom Weighting**: Assigning custom weights to prioritize specific retrievers in the ensemble.
4. **Custom Retrievers**: Integrating custom retrieval logic into the ensemble for domain-specific use cases.
5. **Metadata Filtering**: Refining search results by filtering documents based on metadata.

Each example builds on a common setup (e.g., BM25 and Chroma retrievers) to ensure consistency and reduce redundancy, making them easy to follow and implement in a Kaggle notebook or similar environment. These examples are designed to help users understand and apply advanced retrieval techniques in real-world scenarios.

### Example 1: Combining Sparse and Dense Retrievers for Hybrid Search
This example demonstrates how to combine a sparse retriever (`BM25Retriever`) and a dense retriever (`Chroma Retriever`) using the `EnsembleRetriever`. This hybrid approach leverages the strengths of both keyword-based and semantic search.

In [ ]:
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# Sample documents
doc_list_1 = [
    "I like apples",
    "I like oranges",
    "Apples and oranges are fruits",
]

doc_list_2 = [
    "You like apples",
    "You like oranges",
]

# Initialize BM25 retriever (sparse)
bm25_retriever = BM25Retriever.from_texts(
    doc_list_1, metadatas=[{"source": 1}] * len(doc_list_1)
)
bm25_retriever.k = 2

# Initialize Chroma retriever (dense)
chroma_vectorstore = Chroma.from_texts(
    doc_list_2, embed, metadatas=[{"source": 2}] * len(doc_list_2)
)
chroma_retriever = chroma_vectorstore.as_retriever(search_kwargs={"k": 2})

# Initialize EnsembleRetriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, chroma_retriever], weights=[0.5, 0.5]
)

# Retrieve documents
docs = ensemble_retriever.invoke("apples")
print(docs)

### Example 2: Configuring Retrievers at Runtime
This example shows how to configure the parameters of individual retrievers (e.g., adjusting the number of documents to retrieve) at runtime using `ConfigurableField`.

In [ ]:
from langchain_core.runnables import ConfigurableField

# Make the Chroma retriever configurable
chroma_retriever = chroma_vectorstore.as_retriever(
    search_kwargs={"k": 2}
).configurable_fields(
    search_kwargs=ConfigurableField(
        id="search_kwargs_chroma",
        name="Search Kwargs",
        description="The search kwargs to use",
    )
)

# Reinitialize EnsembleRetriever with the configurable Chroma retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, chroma_retriever], weights=[0.5, 0.5]
)

# Configure Chroma retriever at runtime
config = {"configurable": {"search_kwargs_chroma": {"k": 1}}}
docs = ensemble_retriever.invoke("apples", config=config)
print(docs)

### Example 3: Adjusting Retriever Weights for Custom Prioritization
This example demonstrates how to adjust the weights assigned to each retriever in the ensemble to prioritize one retriever over another.

In [ ]:
# Reinitialize EnsembleRetriever with custom weights
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, chroma_retriever], weights=[0.7, 0.3]
)

# Retrieve documents
docs = ensemble_retriever.invoke("oranges")
print(docs)

### Example 4: Combining Multiple Retrievers for Enhanced Results
This example demonstrates how to combine more than two retrievers (e.g., BM25, Chroma, and a custom retriever) to further enhance retrieval performance.

In [ ]:
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

# Define a custom retriever that returns Document objects
def custom_retriever(query: str) -> list:
    return [
        Document(
            page_content="Custom document",
            metadata={"source": "custom"}
        )
    ]

# Wrap the custom retriever in a RunnableLambda
custom_retriever_runnable = RunnableLambda(custom_retriever)

# Initialize EnsembleRetriever with multiple retrievers
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, chroma_retriever, custom_retriever_runnable],
    weights=[0.4, 0.4, 0.2],
)

# Retrieve documents
docs = ensemble_retriever.invoke("apples")
print(docs)

### Example 5: Using EnsembleRetriever with Metadata Filtering
This example demonstrates how to use metadata filtering with the `EnsembleRetriever` to refine search results.

In [ ]:
# Reinitialize Chroma retriever with metadata filtering
chroma_retriever = chroma_vectorstore.as_retriever(
    search_kwargs={"k": 2, "filter": {"source": 2}}
)

# Reinitialize EnsembleRetriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, chroma_retriever], weights=[0.5, 0.5]
)

# Retrieve documents
docs = ensemble_retriever.invoke("apples")
print(docs)

## Conclusion

The **EnsembleRetriever** is a versatile and powerful tool for enhancing information retrieval systems. By combining the strengths of sparse and dense retrievers, it addresses the limitations of individual retrieval methods and provides a more robust solution for complex search tasks. Its ability to rerank results using the Reciprocal Rank Fusion algorithm ensures that the most relevant documents are prioritized, while its support for runtime configuration and custom weights makes it highly adaptable to various use cases.

Whether you're building a chatbot, a recommendation system, or a knowledge base, the EnsembleRetriever offers a flexible and effective way to improve search relevance and accuracy. Its hybrid approach makes it particularly valuable in scenarios where both keyword-based and semantic-based retrieval are required.